## Quora Insicere Questions Classification

**Experiment 1:**
- Without pretrained + LSTM

**Experiment 2:**
- With pretrained(Google news pretrained Embeddings ) + LSTM

In [ ]:
### importing the libraries 
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from gensim.models import KeyedVectors
import re ## Regular expresssions
from nltk import word_tokenize
from sklearn import metrics
from gensim.models import KeyedVectors
import operator
import gc

In [ ]:
### reading the data files
test_data=pd.read_csv("../input/quora-insincere-questions-classification/test.csv")
train_data=pd.read_csv("../input/quora-insincere-questions-classification/train.csv")

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
### splitting the dataset in to the training set and validation set 
train,val=train_test_split(train_data,test_size=0.2,stratify=train_data.target,random_state=123)
print("Shape of the Training set :",train.shape)
print("Shape of the Validation set :",val.shape)

In [ ]:
%%time
### unzipping all the pretrained embeddings
!unzip ../input/quora-insincere-questions-classification/embeddings.zip

### Experiment 1: With out Pretrained Embeddings + LSTM
- here we will train the embeddings From scratch 

**steps to do this**
        - firstly preprocess the data
        - Get vocabulary of training corpus
        - Convert the text to sequence (one hot formate)
        - pad sequence to maxlength this has to be fixed length
        - we will input the padded sequence to the Embedding layer in the network.

#### step:1 Preprocess the data
**Experiment 1.1 : with out preprocess**
        - here i am not going to preprocess much here
        - becz i am just want the sequence as it was 
        - here we do just normalization i.e converting from uppercase to lowercase letters

**Experiment 1.2 : With Preprocess**

    - converting the number to unique code
    - Applying the contractions 
    - removing the punctuation marks
    - Normalizing the text
    

In [ ]:
print(train.question_text[0])

In [ ]:
contractions={"I'm": 'I am',
 "I'm'a": 'I am about to',
 "I'm'o": 'I am going to',
 "I've": 'I have',
 "I'll": 'I will',
 "I'll've": 'I will have',
 "I'd": 'I would',
 "I'd've": 'I would have',
 'Whatcha': 'What are you',
 "amn't": 'am not',
 "ain't": 'are not',
 "aren't": 'are not',
 "'cause": 'because',
 "can't": 'can not',
 "can't've": 'can not have',
 "could've": 'could have',
 "couldn't": 'could not',
 "couldn't've": 'could not have',
 "daren't": 'dare not',
 "daresn't": 'dare not',
 "dasn't": 'dare not',
 "didn't": 'did not',
 'didn’t': 'did not',
 "don't": 'do not',
 'don’t': 'do not',
 "doesn't": 'does not',
 "e'er": 'ever',
 "everyone's": 'everyone is',
 'finna': 'fixing to',
 'gimme': 'give me',
 "gon't": 'go not',
 'gonna': 'going to',
 'gotta': 'got to',
 "hadn't": 'had not',
 "hadn't've": 'had not have',
 "hasn't": 'has not',
 "haven't": 'have not',
 "he've": 'he have',
 "he's": 'he is',
 "he'll": 'he will',
 "he'll've": 'he will have',
 "he'd": 'he would',
 "he'd've": 'he would have',
 "here's": 'here is',
 "how're": 'how are',
 "how'd": 'how did',
 "how'd'y": 'how do you',
 "how's": 'how is',
 "how'll": 'how will',
 "isn't": 'is not',
 "it's": 'it is',
 "'tis": 'it is',
 "'twas": 'it was',
 "it'll": 'it will',
 "it'll've": 'it will have',
 "it'd": 'it would',
 "it'd've": 'it would have',
 'kinda': 'kind of',
 "let's": 'let us',
 'luv': 'love',
 "ma'am": 'madam',
 "may've": 'may have',
 "mayn't": 'may not',
 "might've": 'might have',
 "mightn't": 'might not',
 "mightn't've": 'might not have',
 "must've": 'must have',
 "mustn't": 'must not',
 "mustn't've": 'must not have',
 "needn't": 'need not',
 "needn't've": 'need not have',
 "ne'er": 'never',
 "o'": 'of',
 "o'clock": 'of the clock',
 "ol'": 'old',
 "oughtn't": 'ought not',
 "oughtn't've": 'ought not have',
 "o'er": 'over',
 "shan't": 'shall not',
 "sha'n't": 'shall not',
 "shalln't": 'shall not',
 "shan't've": 'shall not have',
 "she's": 'she is',
 "she'll": 'she will',
 "she'd": 'she would',
 "she'd've": 'she would have',
 "should've": 'should have',
 "shouldn't": 'should not',
 "shouldn't've": 'should not have',
 "so've": 'so have',
 "so's": 'so is',
 "somebody's": 'somebody is',
 "someone's": 'someone is',
 "something's": 'something is',
 'sux': 'sucks',
 "that're": 'that are',
 "that's": 'that is',
 "that'll": 'that will',
 "that'd": 'that would',
 "that'd've": 'that would have',
 'em': 'them',
 "there're": 'there are',
 "there's": 'there is',
 "there'll": 'there will',
 "there'd": 'there would',
 "there'd've": 'there would have',
 "these're": 'these are',
 "they're": 'they are',
 "they've": 'they have',
 "they'll": 'they will',
 "they'll've": 'they will have',
 "they'd": 'they would',
 "they'd've": 'they would have',
 "this's": 'this is',
 "those're": 'those are',
 "to've": 'to have',
 'wanna': 'want to',
 "wasn't": 'was not',
 "we're": 'we are',
 "we've": 'we have',
 "we'll": 'we will',
 "we'll've": 'we will have',
 "we'd": 'we would',
 "we'd've": 'we would have',
 "weren't": 'were not',
 "what're": 'what are',
 "what'd": 'what did',
 "what've": 'what have',
 "what's": 'what is',
 "what'll": 'what will',
 "what'll've": 'what will have',
 "when've": 'when have',
 "when's": 'when is',
 "where're": 'where are',
 "where'd": 'where did',
 "where've": 'where have',
 "where's": 'where is',
 "which's": 'which is',
 "who're": 'who are',
 "who've": 'who have',
 "who's": 'who is',
 "who'll": 'who will',
 "who'll've": 'who will have',
 "who'd": 'who would',
 "who'd've": 'who would have',
 "why're": 'why are',
 "why'd": 'why did',
 "why've": 'why have',
 "why's": 'why is',
 "will've": 'will have',
 "won't": 'will not',
 "won't've": 'will not have',
 "would've": 'would have',
 "wouldn't": 'would not',
 "wouldn't've": 'would not have',
 "y'all": 'you all',
 "y'all're": 'you all are',
 "y'all've": 'you all have',
 "y'all'd": 'you all would',
 "y'all'd've": 'you all would have',
 "you're": 'you are',
 "you've": 'you have',
 "you'll've": 'you shall have',
 "you'll": 'you will',
 "you'd": 'you would',
 "you'd've": 'you would have',
 'jan.': 'january',
 'feb.': 'february',
 'mar.': 'march',
 'apr.': 'april',
 'jun.': 'june',
 'jul.': 'july',
 'aug.': 'august',
 'sep.': 'september',
 'oct.': 'october',
 'nov.': 'november',
 'dec.': 'december',
 'I’m': 'I am',
 'I’m’a': 'I am about to',
 'I’m’o': 'I am going to',
 'I’ve': 'I have',
 'I’ll': 'I will',
 'I’ll’ve': 'I will have',
 'I’d': 'I would',
 'I’d’ve': 'I would have',
 'amn’t': 'am not',
 'ain’t': 'are not',
 'aren’t': 'are not',
 '’cause': 'because',
 'can’t': 'can not',
 'can’t’ve': 'can not have',
 'could’ve': 'could have',
 'couldn’t': 'could not',
 'couldn’t’ve': 'could not have',
 'daren’t': 'dare not',
 'daresn’t': 'dare not',
 'dasn’t': 'dare not',
 'doesn’t': 'does not',
 'e’er': 'ever',
 'everyone’s': 'everyone is',
 'gon’t': 'go not',
 'hadn’t': 'had not',
 'hadn’t’ve': 'had not have',
 'hasn’t': 'has not',
 'haven’t': 'have not',
 'he’ve': 'he have',
 'he’s': 'he is',
 'he’ll': 'he will',
 'he’ll’ve': 'he will have',
 'he’d': 'he would',
 'he’d’ve': 'he would have',
 'here’s': 'here is',
 'how’re': 'how are',
 'how’d': 'how did',
 'how’d’y': 'how do you',
 'how’s': 'how is',
 'how’ll': 'how will',
 'isn’t': 'is not',
 'it’s': 'it is',
 '’tis': 'it is',
 '’twas': 'it was',
 'it’ll': 'it will',
 'it’ll’ve': 'it will have',
 'it’d': 'it would',
 'it’d’ve': 'it would have',
 'let’s': 'let us',
 'ma’am': 'madam',
 'may’ve': 'may have',
 'mayn’t': 'may not',
 'might’ve': 'might have',
 'mightn’t': 'might not',
 'mightn’t’ve': 'might not have',
 'must’ve': 'must have',
 'mustn’t': 'must not',
 'mustn’t’ve': 'must not have',
 'needn’t': 'need not',
 'needn’t’ve': 'need not have',
 'ne’er': 'never',
 'o’': 'of',
 'o’clock': 'of the clock',
 'ol’': 'old',
 'oughtn’t': 'ought not',
 'oughtn’t’ve': 'ought not have',
 'o’er': 'over',
 'shan’t': 'shall not',
 'sha’n’t': 'shall not',
 'shalln’t': 'shall not',
 'shan’t’ve': 'shall not have',
 'she’s': 'she is',
 'she’ll': 'she will',
 'she’d': 'she would',
 'she’d’ve': 'she would have',
 'should’ve': 'should have',
 'shouldn’t': 'should not',
 'shouldn’t’ve': 'should not have',
 'so’ve': 'so have',
 'so’s': 'so is',
 'somebody’s': 'somebody is',
 'someone’s': 'someone is',
 'something’s': 'something is',
 'that’re': 'that are',
 'that’s': 'that is',
 'that’ll': 'that will',
 'that’d': 'that would',
 'that’d’ve': 'that would have',
 'there’re': 'there are',
 'there’s': 'there is',
 'there’ll': 'there will',
 'there’d': 'there would',
 'there’d’ve': 'there would have',
 'these’re': 'these are',
 'they’re': 'they are',
 'they’ve': 'they have',
 'they’ll': 'they will',
 'they’ll’ve': 'they will have',
 'they’d': 'they would',
 'they’d’ve': 'they would have',
 'this’s': 'this is',
 'those’re': 'those are',
 'to’ve': 'to have',
 'wasn’t': 'was not',
 'we’re': 'we are',
 'we’ve': 'we have',
 'we’ll': 'we will',
 'we’ll’ve': 'we will have',
 'we’d': 'we would',
 'we’d’ve': 'we would have',
 'weren’t': 'were not',
 'what’re': 'what are',
 'what’d': 'what did',
 'what’ve': 'what have',
 'what’s': 'what is',
 'what’ll': 'what will',
 'what’ll’ve': 'what will have',
 'when’ve': 'when have',
 'when’s': 'when is',
 'where’re': 'where are',
 'where’d': 'where did',
 'where’ve': 'where have',
 'where’s': 'where is',
 'which’s': 'which is',
 'who’re': 'who are',
 'who’ve': 'who have',
 'who’s': 'who is',
 'who’ll': 'who will',
 'who’ll’ve': 'who will have',
 'who’d': 'who would',
 'who’d’ve': 'who would have',
 'why’re': 'why are',
 'why’d': 'why did',
 'why’ve': 'why have',
 'why’s': 'why is',
 'will’ve': 'will have',
 'won’t': 'will not',
 'won’t’ve': 'will not have',
 'would’ve': 'would have',
 'wouldn’t': 'would not',
 'wouldn’t’ve': 'would not have',
 'y’all': 'you all',
 'y’all’re': 'you all are',
 'y’all’ve': 'you all have',
 'y’all’d': 'you all would',
 'y’all’d’ve': 'you all would have',
 'you’re': 'you are',
 'you’ve': 'you have',
 'you’ll’ve': 'you shall have',
 'you’ll': 'you will',
 'you’d': 'you would',
 'you’d’ve': 'you would have'}

def contraction_fix(word):
    try:
        a=contractions[word]
    except KeyError:
        a=word
    return a

In [ ]:
def Preprocess(doc):
    corpus=[]
    for text in tqdm(doc):
        text=" ".join([contraction_fix(w) for w in text.split()])
        text=re.sub(r'[^a-z0-9A-Z]'," ",text)
        text=re.sub(r'[0-9]{1}',"#",text)
        text=re.sub(r'[0-9]{2}','##',text)
        text=re.sub(r'[0-9]{3}','###',text)
        text=re.sub(r'[0-9]{4}','####',text)
        text=re.sub(r'[0-9]{5,}','#####',text)
        corpus.append(text)
    return corpus

In [ ]:
## defining the vocabular function
def get_vocab(corpus):
    vocab={}
    for text in tqdm(corpus):
        for word in text.split():
            try:
                vocab[word]+=1
            except KeyError:
                vocab[word]=1
    vocab=dict(sorted(vocab.items(),reverse=True ,key=lambda item: item[1]))
    return vocab

### Expo 1.1 : without preprocessing + without pretrained Embeddings

- here we are not going to do any preproess on text . just using the raw text.
- Learning Embeddings during Trainig phase.

In [ ]:
### importing the libraire
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D,GRU
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model,load_model
from keras.preprocessing.sequence import pad_sequences
from keras.utils import plot_model
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.optimizers import Adam
import tensorflow as tf

In [ ]:
### defining the parameter
max_feat=30000
max_len=40
feat_vec=300

In [ ]:
### gettting the index for the each word in vocabulary
def get_word_index(vocab):
    word_index=dict((w,i+1) for i,w in enumerate(vocab.keys()))
    return word_index
def fit_one_hot(word_index,corpus):
    sent=[]
    for text in tqdm(corpus):
        li=[]
        for word in text.split():
            try:
                li.append(word_index[word])
            except KeyError:
                li.append(0)
        sent.append(li)
    return sent

In [ ]:
vocab=get_vocab(train.question_text)
top_feat=dict(list(vocab.items())[:max_feat])
word_index=get_word_index(top_feat)
encoded_docs=fit_one_hot(word_index,train.question_text)
padded_doc=pad_sequences(encoded_docs,maxlen=max_len,padding="post")
vocab_size=len(word_index)

In [ ]:
### getting the Validation data
val_encodes=fit_one_hot(word_index,val.question_text)
val_padded_doc=pad_sequences(val_encodes,maxlen=max_len,padding="post")

### Building the Model

In [ ]:
### defining the parameter
max_feat=30000
max_len=40
feat_vec=300

In [ ]:
%%time
### preprocessing the text
train_text=Preprocess(train.question_text)
val_text=Preprocess(val.question_text)


vocab=get_vocab(train_text)
top_feat=dict(list(vocab.items())[:max_feat])
word_index=get_word_index(top_feat)

### encoding the training set
encoded_docs=fit_one_hot(word_index,train_text)
padded_doc=pad_sequences(encoded_docs,maxlen=max_len,padding="post")

### encoding the Validation set
encoded_docs=fit_one_hot(word_index,val_text)
val_padded_doc=pad_sequences(encoded_docs,maxlen=max_len,padding="post")

### Building the Model

In [ ]:
inp = Input(shape=(max_len,))
x = Embedding(max_feat+1, feat_vec)(inp)
x = Bidirectional(GRU(128, return_sequences=True))(x)
x = Conv1D(64,5,activation="relu")(x)
x = GlobalMaxPool1D()(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.2)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)


print(model.summary())

In [ ]:
### defining some callbacks
opt=Adam(learning_rate=0.002)
bin_loss=tf.keras.losses.BinaryCrossentropy(
                                            from_logits=False, 
                                            label_smoothing=0.2,
                                            name='binary_crossentropy'
                                        )

## defining the call backs
early_stopping=tf.keras.callbacks.EarlyStopping(
                                                monitor="val_loss",
                                                patience=3,
                                                mode="min",
                                                restore_best_weights=True
                                              )
### Now reducing the learning rate when the model is not improvinig 
reduce_lr=tf.keras.callbacks.ReduceLROnPlateau(
                                                monitor="val_loss",
                                                factor=0.2,
                                                patience=2,
                                                verbose=1,
                                                mode="auto"
                                            )

my_callbacks=[early_stopping,reduce_lr]



## Experiment 2: Pretrained Embeddings + LSTM
- Here I am using the GOOGle News as pretrained Embeddings

### HOw much percentage that covered of Entire Corpus
- Here we check for the each pretrained Embeddings 
- Do processing that will conver much embeddings


#### 2.1 LOading and Checking the Pretrained Embeddings (GOOGLE NEWS DATA) 

In [ ]:
%%time
### Loading the Google News Pretrained Embeddings
file_name="./GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin"
model_embed=KeyedVectors.load_word2vec_format(file_name,binary=True)

### Building the Vocabulary 
def vocab_build(corpus):
    vocab={}
    for text in tqdm(corpus):
        for word in text.split():
            try:
                vocab[word]+=1
            except KeyError:
                vocab[word]=1
    return vocab


### Checking the Vocabulary That how much percentage of Vocabulary was covered 
def check_voc(vocab,model):
    embed_words=[]
    out_vocab={}
    total_words=0
    total_text=0
    for i in tqdm(vocab):
        try:
            vec=model[i]
            embed_words.append(vec)
            total_words+=vocab[i]
        except KeyError:
            out_vocab[i]=vocab[i]
            total_text+=vocab[i]
    print("The {:.2f}% of vocabularies have Covered of corpus".format(100*len(embed_words)/len(vocab)))
    print("The {:.2f}% of total text had coverded ".format((100*total_words/(total_words+total_text))))
    return out_vocab

In [ ]:
### here we are concatenating the both the training and validation dataset
### building the vocabulary and checking coverage of vocabulary
total_text=pd.concat([train_data.question_text,test_data.question_text])
vocabulary=vocab_build(total_text)
oov=check_voc(vocabulary,model_embed)

## Lets Analyze
- We observed that there will less amount of vocabulary was covered
- now we wiil analyze the out of vocabulary.
- to increase the coverage we will do preprocess the data 

In [ ]:
sort_oov=dict(sorted(oov.items(), key=operator.itemgetter(1),reverse=True))
dict(list(sort_oov.items())[:50])

**Observations**
- some of the puncutation marks are not in pretrained embeedings
- May be the raw numbers are in the form of other type
- Question mark is followed by the word so we need to seperate that and that too question mark doesn't have a embedding.
- Some preprosition too doesn't have the embeddings like (a,to,of and)

In [ ]:
del oov,vocabulary,sort_oov
gc.collect()

In [ ]:
### lets try to resolve above problems by preprocessing it 
pre_text=Preprocess(total_text)
vocabulary=vocab_build(pre_text)
oov=check_voc(vocabulary,model_embed)

In [ ]:
sort_oov=dict(sorted(oov.items(), key=operator.itemgetter(1),reverse=True))
dict(list(sort_oov.items())[:50])

- here i am ending this here .

**Furthur Preprocessing is Required**
- Spell correction mainly 
- Some word doesn't have a embeddings in a pretrained models

In [ ]:
del oov,pre_text,sort_oov,total_text
gc.collect()

In [ ]:
vocab_size=len(vocabulary)+1
max_len=40

word_index=get_word_index(vocabulary)
### preprocess the data
train_text=Preprocess(train.question_text)
val_text=Preprocess(val.question_text)
test_text=Preprocess(test_data.question_text)

### encoding the training data
encodes=fit_one_hot(word_index,train_text)
train_padded=pad_sequences(encodes,maxlen=max_len,padding="post")

### encoding the validation_data
encodes_=fit_one_hot(word_index,val_text)
val_padded=pad_sequences(encodes_,maxlen=max_len,padding="post")

### encoding the testing_data
encodes__=fit_one_hot(word_index,test_text)
test_padded=pad_sequences(encodes__,maxlen=max_len,padding="post")



### Constructing the Embedding vector

In [ ]:
## step 3: constructinng embedding matrix for the corpus vocabulary using the pretrained embeddings:
## here each row will have the emedding vector for each unique word
count=0
embedding_mat=np.zeros((vocab_size,300))
for word,i in tqdm(word_index.items()):
    try:
        vec=model_embed[word]
        embedding_mat[i]=vec
    except KeyError:
        count+=1
        continue

print("Number of Out of Vocabulary",count)

In [ ]:
inp = Input(shape=(max_len,))
x = Embedding(vocab_size,300,weights=[embedding_mat],input_length=max_len,trainable=False)(inp)
x = Bidirectional(LSTM(128, return_sequences=True))(x)
x = Conv1D(64,3,activation="relu")(x)
x = GlobalMaxPool1D()(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.2)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)


print(model.summary())

In [ ]:
opt=Adam(learning_rate=0.001)
bin_loss=tf.keras.losses.BinaryCrossentropy(
                                            from_logits=False, 
                                            label_smoothing=0,
                                            name='binary_crossentropy'
                                        )

## defining the call backs
early_stopping=tf.keras.callbacks.EarlyStopping(
                                                monitor="val_loss",
                                                patience=3,
                                                mode="min",
                                                restore_best_weights=True
                                              )
### Now reducing the learning rate when the model is not improvinig 
reduce_lr=tf.keras.callbacks.ReduceLROnPlateau(
                                                monitor="val_loss",
                                                factor=0.2,
                                                patience=2,
                                                verbose=1,
                                                mode="auto"
                                            )

my_callbacks=[early_stopping,reduce_lr]



In [ ]:
model.compile(loss=bin_loss, optimizer=opt, metrics=['accuracy'])
history=model.fit(train_padded, train.target, batch_size=512, epochs=30, validation_data=(val_padded, val.target),callbacks=my_callbacks)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
y_pre=model.predict(val_padded)
for thresh in np.arange(0.1,0.5,0.01):
    print("threshold {0:2.2f} f1 score:{1:2.3f}".format(thresh,metrics.f1_score(val.target,(y_pre>thresh).astype(int))))


In [ ]:
### INference the Test data

threshold=0.36
y_test_pre=model.predict(test_padded)
y_test_pre=(y_test_pre>thresh).astype(int)

### Creating the submission File
submit=pd.DataFrame()
submit["qid"]=test_data.qid
submit["prediction"]=y_test_pre
submit.to_csv("submission.csv",index=False)

In [ ]:
model.save('hamza.h5')